In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=c44862b4b1785b56df51d584c52df787a698b38580e485613c36666ff26d48fb
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pandas as pd

users = pd.read_table('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user',
                      sep='|', index_col='user_id')
users.head()



,age,gender,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkBy').getOrCreate()
users_df = spark.createDataFrame(data=users)

In [4]:

#step5




def male_ratio_per_occupation(df):


  from pyspark.sql.functions import col, count, when
  male_ratio = users_df.groupBy('occupation') \
                      .agg(count('*').alias('total'),
                           count(when(col('gender') == 'M', True)).alias('male_count')) \
                      .withColumn('male_ratio', col('male_count') / col('total')) \
                      .orderBy(col('male_ratio').desc())
  return male_ratio

male_ratio_df = male_ratio_per_occupation(users_df)
male_ratio_df.show()


+-------------+-----+----------+-------------------+
|   occupation|total|male_count|         male_ratio|
+-------------+-----+----------+-------------------+
|       doctor|    7|         7|                1.0|
|     engineer|   67|        65| 0.9701492537313433|
|   technician|   27|        26| 0.9629629629629629|
|      retired|   14|        13| 0.9285714285714286|
|   programmer|   66|        60| 0.9090909090909091|
|    executive|   32|        29|            0.90625|
|    scientist|   31|        28| 0.9032258064516129|
|entertainment|   18|        16| 0.8888888888888888|
|       lawyer|   12|        10| 0.8333333333333334|
|     salesman|   12|         9|               0.75|
|     educator|   95|        69| 0.7263157894736842|
|      student|  196|       136| 0.6938775510204082|
|        other|  105|        69| 0.6571428571428571|
|    marketing|   26|        16| 0.6153846153846154|
|       writer|   45|        26| 0.5777777777777777|
|         none|    9|         5| 0.55555555555

In [ ]:
def male_ratio_per_occupation(users_df):

  from pyspark.sql.functions import col, count, when
  male_ratio = users_df.groupBy('occupation') \
                      .agg(count('*').alias('total'),
                           count(when(col('gender') == 'M', True)).alias('male_count')) \
                      .withColumn('male_ratio', col('male_count') / col('total')) \
                      .orderBy(col('male_ratio').desc())
  return male_ratio

male_ratio_df = male_ratio_per_occupation(users_df)
male_ratio_df.show()

+-------------+-----+----------+-------------------+
|   occupation|total|male_count|         male_ratio|
+-------------+-----+----------+-------------------+
|       doctor|    7|         7|                1.0|
|     engineer|   67|        65| 0.9701492537313433|
|   technician|   27|        26| 0.9629629629629629|
|      retired|   14|        13| 0.9285714285714286|
|   programmer|   66|        60| 0.9090909090909091|
|    executive|   32|        29|            0.90625|
|    scientist|   31|        28| 0.9032258064516129|
|entertainment|   18|        16| 0.8888888888888888|
|       lawyer|   12|        10| 0.8333333333333334|
|     salesman|   12|         9|               0.75|
|     educator|   95|        69| 0.7263157894736842|
|      student|  196|       136| 0.6938775510204082|
|        other|  105|        69| 0.6571428571428571|
|    marketing|   26|        16| 0.6153846153846154|
|       writer|   45|        26| 0.5777777777777777|
|         none|    9|         5| 0.55555555555

In [ ]:
users_df.printSchema()

root
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- zip_code: string (nullable = true)



In [16]:
def mean_age_by_occupation(df):
 return df.groupBy("occupation").agg({'age': 'mean'})


 result = mean_age_by_occupation(users_df)
 result.show()

In [15]:
users_df.groupBy("occupation").agg({'age': 'mean'}).show()

+-------------+------------------+
|   occupation|          avg(age)|
+-------------+------------------+
|    librarian|              40.0|
|      retired| 63.07142857142857|
|       lawyer|             36.75|
|         none|26.555555555555557|
|       writer| 36.31111111111111|
|   programmer|33.121212121212125|
|    marketing| 37.61538461538461|
|        other|34.523809523809526|
|    executive|          38.71875|
|    scientist| 35.54838709677419|
|      student|22.081632653061224|
|     salesman|35.666666666666664|
|       artist|31.392857142857142|
|   technician|33.148148148148145|
|administrator| 38.74683544303797|
|     engineer| 36.38805970149254|
|   healthcare|           41.5625|
|     educator| 42.01052631578948|
|entertainment| 29.22222222222222|
|    homemaker| 32.57142857142857|
+-------------+------------------+
only showing top 20 rows



In [ ]:
import pyspark.sql.functions as F

def calculate_male_ratio_by_occupation(df):


  # Create a column to indicate whether the gender is male
  df = df.withColumn("is_male", F.when(F.col("gender") == "M", 1).otherwise(0))

  # Calculate the male ratio per occupation
  male_ratio_df = df.groupBy("occupation").agg(
      F.sum("is_male").alias("total_males"),
      F.count("*").alias("total_count")
  ).withColumn("male_ratio", F.col("total_males") / F.col("total_count"))

  # Sort by male ratio in descending order
  male_ratio_df = male_ratio_df.orderBy(F.desc("male_ratio"))

  return male_ratio_df


In [6]:
# step6
from pyspark.sql import functions as F
def min_max_age_per_occupation(df):

  age_range = users_df.groupBy('occupation') \
                      .agg(F.min('age').alias('min_age'),
                           F.max('age').alias('max_age'))
  return age_range

age_range_df = min_max_age_per_occupation(users_df)
age_range_df.show()


+-------------+-------+-------+
|   occupation|min_age|max_age|
+-------------+-------+-------+
|    librarian|     23|     69|
|      retired|     51|     73|
|       lawyer|     21|     53|
|         none|     11|     55|
|       writer|     18|     60|
|   programmer|     20|     63|
|    marketing|     24|     55|
|        other|     13|     64|
|    executive|     22|     69|
|    scientist|     23|     55|
|      student|      7|     42|
|     salesman|     18|     66|
|       artist|     19|     48|
|   technician|     21|     55|
|administrator|     21|     70|
|     engineer|     22|     70|
|   healthcare|     22|     62|
|     educator|     23|     63|
|entertainment|     15|     50|
|    homemaker|     20|     50|
+-------------+-------+-------+
only showing top 20 rows



In [7]:
# step 7

def mean_age_by_occupation_gender(df):

  mean_age_df = users_df.groupBy("occupation","gender").agg({'age': 'mean'})
  return mean_age_df

mean_age_df = mean_age_by_occupation_gender(users_df)
mean_age_df.show()


+-------------+------+------------------+
|   occupation|gender|          avg(age)|
+-------------+------+------------------+
|   technician|     M| 32.96153846153846|
|     educator|     F| 39.11538461538461|
|       lawyer|     F|              39.5|
|       lawyer|     M|              36.2|
|      student|     F|             20.75|
|   healthcare|     F| 39.81818181818182|
|administrator|     M| 37.16279069767442|
|    marketing|     M|            37.875|
|    homemaker|     F|34.166666666666664|
|       artist|     F|30.307692307692307|
|         none|     F|              36.5|
|       doctor|     M| 43.57142857142857|
|       writer|     F| 37.63157894736842|
|     educator|     M| 43.10144927536232|
|    scientist|     M| 36.32142857142857|
|       writer|     M| 35.34615384615385|
|        other|     F| 35.47222222222222|
|   healthcare|     M|              45.4|
|    scientist|     F|28.333333333333332|
|         none|     M|              18.6|
+-------------+------+------------

In [8]:
# step 8
from pyspark.sql.functions import col, count, sum
def gender_percentage_per_occupation(df):

  gender_counts = users_df.groupBy('occupation', 'gender').agg(count('*').alias('count'))
  occupation_totals = gender_counts.groupBy('occupation').agg(sum('count').alias('total'))
  gender_percentages = gender_counts.join(occupation_totals, 'occupation') \
                                   .withColumn('percentage', (col('count') / col('total')) * 100) \
                                   .select('occupation', 'gender', 'percentage')
  return gender_percentages

gender_percentages_df = gender_percentage_per_occupation(users_df)
gender_percentages_df.show()


+-------------+------+------------------+
|   occupation|gender|        percentage|
+-------------+------+------------------+
|   technician|     M| 96.29629629629629|
|     educator|     F|27.368421052631582|
|       lawyer|     F|16.666666666666664|
|       lawyer|     M| 83.33333333333334|
|      student|     F|30.612244897959183|
|   healthcare|     F|             68.75|
|administrator|     M| 54.43037974683544|
|    marketing|     M| 61.53846153846154|
|    homemaker|     F| 85.71428571428571|
|       artist|     F| 46.42857142857143|
|         none|     F| 44.44444444444444|
|       doctor|     M|             100.0|
|       writer|     F| 42.22222222222222|
|     educator|     M| 72.63157894736842|
|    scientist|     M| 90.32258064516128|
|       writer|     M| 57.77777777777777|
|        other|     F|34.285714285714285|
|   healthcare|     M|             31.25|
|    scientist|     F|  9.67741935483871|
|         none|     M| 55.55555555555556|
+-------------+------+------------

In [ ]:
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName('SparkBy').getOrCreate()
#users_df = spark.createDataFrame(data=users)


import pandas as pd

users = pd.read_table('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/u.user',
                      sep='|', index_col='user_id')
users.head()


spark = SparkSession.builder.appName('SparkBy').getOrCreate()
users_df = spark.createDataFrame(data=users)


gender_perct_occupation = s.gender_percentage_per_occupation(df)
print(f"percentage:{gender_perct_occupation}")


mean_age_gender =s.mean_age_by_occupation_gender(df)
print(f"mean_age_gender:{mean_age_gender}")

age_range = s.min_max_age_per_occupation(df)
print(f"age_range:{age_range}")

male_ratio_df = s.male_ratio_per_occupation(df)
print(f"male_ratio_df:{male_ratio_df}")






mean_age_occupation =s.calculate_mean_age_by_occupation(df)
print(f"mean_age_occupation:{mean_age_occupation}")






In [ ]:
from pyspark.sql import functions as F

def calculate_mean_age_by_occupation(df):


  mean_age_df = df.groupBy("occupation").agg(F.mean("age").alias("mean_age"))
  return mean_age_df

result = calculate_mean_age_by_occupation(users_df)
result.show()

+-------------+------------------+
|   occupation|          mean_age|
+-------------+------------------+
|    librarian|              40.0|
|      retired| 63.07142857142857|
|       lawyer|             36.75|
|         none|26.555555555555557|
|       writer| 36.31111111111111|
|   programmer|33.121212121212125|
|    marketing| 37.61538461538461|
|        other|34.523809523809526|
|    executive|          38.71875|
|    scientist| 35.54838709677419|
|      student|22.081632653061224|
|     salesman|35.666666666666664|
|       artist|31.392857142857142|
|   technician|33.148148148148145|
|administrator| 38.74683544303797|
|     engineer| 36.38805970149254|
|   healthcare|           41.5625|
|     educator| 42.01052631578948|
|entertainment| 29.22222222222222|
|    homemaker| 32.57142857142857|
+-------------+------------------+
only showing top 20 rows

